In [34]:
import pandas as pd

In [35]:
exchange=pd.read_csv('../staging_1/exchange/exchange.csv')

In [36]:
order_items=pd.read_csv('../staging_1/order_items/order_items.csv')

In [37]:
exchange.head()

,Timestamp,EGP Rate,Source
0,2024-04-30 13:33:36.919465,47.924824,API


In [38]:
order_items.head()

,order_id,item_id,product_id,quantity,list_price,discount,extract_time,source
0,1,1.0,20,1,599.99,0.20,2024-04-30 13:07:13.468028,postgres
1,1,2.0,8,2,1799.99,0.07,2024-04-30 13:07:13.468028,postgres
2,1,3.0,10,2,1549.00,0.05,2024-04-30 13:07:13.468028,postgres
3,1,4.0,16,2,599.99,0.05,2024-04-30 13:07:13.468028,postgres
4,1,5.0,4,1,2899.99,0.20,2024-04-30 13:07:13.468028,postgres


In [39]:
import pandas as pd


# Create a new column with a constant value to merge on
exchange['key'] = 1
order_items['key'] = 1

# Perform the cross join
cross_join_df = pd.merge(exchange, order_items, on='key')

cross_join_df.drop(columns=['key','Timestamp','Source'], inplace=True)

In [40]:
cross_join_df.shape

(4601, 9)

In [41]:
exchange.drop(columns='key', inplace=True)
order_items.drop(columns='key', inplace=True)

In [42]:

# Calculate "list_price_egp" column
cross_join_df['list_price_egp'] = cross_join_df['list_price'] * cross_join_df['EGP Rate']

cross_join_df.head()

,EGP Rate,order_id,item_id,product_id,quantity,list_price,discount,extract_time,source,list_price_egp
0,47.924824,1,1.0,20,1,599.99,0.20,2024-04-30 13:07:13.468028,postgres,28754.415152
1,47.924824,1,2.0,8,2,1799.99,0.07,2024-04-30 13:07:13.468028,postgres,86264.203952
2,47.924824,1,3.0,10,2,1549.00,0.05,2024-04-30 13:07:13.468028,postgres,74235.552376
3,47.924824,1,4.0,16,2,599.99,0.05,2024-04-30 13:07:13.468028,postgres,28754.415152
4,47.924824,1,5.0,4,1,2899.99,0.20,2024-04-30 13:07:13.468028,postgres,138981.510352


In [43]:
import os

# Define the parent directory path
parent_directory = '../staging_2'

# Define the subdirectory path
subdirectory = 'order_items'

# Create the parent directory if it doesn't exist
if not os.path.exists(parent_directory):
    os.makedirs(parent_directory)
    print(f"Parent directory '{parent_directory}' created.")

# Create the subdirectory if it doesn't exist
if not os.path.exists(os.path.join(parent_directory, subdirectory)):
    os.makedirs(os.path.join(parent_directory, subdirectory))
    print(f"Subdirectory '{subdirectory}' created.")

# Define the file path where you want to save the CSV file
file_path = os.path.join(parent_directory, subdirectory, 'order_items.csv')

# Save the DataFrame as a CSV file
cross_join_df.to_csv(file_path, index=False)

print(f"CSV file saved successfully in the '{subdirectory}' subfolder within the '{parent_directory}' folder.")


CSV file saved successfully in the 'order_items' subfolder within the '../staging_2' folder.


In [44]:
orders=pd.read_csv('../staging_1/orders/orders.csv')

In [45]:
orders.head()

,order_id,customer_id,order_status,order_date,required_date,shipped_date,store_id,staff_id,extract_time,source
0,1,259,4,2016-01-01,2016-01-03 00:00:00.000000000,2016-01-03 00:00:00.000000000,1,2,2024-04-30 13:07:13.382187,postgres
1,2,1212,4,2016-01-01,2016-01-04 00:00:00.000000000,2016-01-03 00:00:00.000000000,2,6,2024-04-30 13:07:13.382187,postgres
2,3,523,4,2016-01-02,2016-01-05 00:00:00.000000000,2016-01-03 00:00:00.000000000,2,7,2024-04-30 13:07:13.382187,postgres
3,4,175,4,2016-01-03,2016-01-04 00:00:00.000000000,2016-01-05 00:00:00.000000000,1,3,2024-04-30 13:07:13.382187,postgres
4,5,1324,4,2016-01-03,2016-01-06 00:00:00.000000000,2016-01-06 00:00:00.000000000,2,6,2024-04-30 13:07:13.382187,postgres


In [46]:
# Convert date columns to datetime format
orders['order_date'] = pd.to_datetime(orders['order_date'],format='%Y/%m/%d')
orders['required_date'] = pd.to_datetime(orders['required_date'],format='%Y/%m/%d')
orders['shipped_date'] = pd.to_datetime(orders['shipped_date'],format='%Y/%m/%d')
orders['extract_time'] = pd.to_datetime(orders['extract_time'],format='%Y/%m/%d')

# Calculate late_delivery column
orders['late_delivery'] = orders['shipped_date'] > orders['required_date']

# Calculate latency_days column
orders['latency_days'] = (orders['shipped_date'] - orders['required_date']).dt.days
orders.loc[~orders['late_delivery'], 'latency_days'] = None

In [47]:
orders.head()

,order_id,customer_id,order_status,order_date,required_date,shipped_date,store_id,staff_id,extract_time,source,late_delivery,latency_days
0,1,259,4,2016-01-01,2016-01-03,2016-01-03,1,2,2024-04-30 13:07:13.382187,postgres,False,NaN
1,2,1212,4,2016-01-01,2016-01-04,2016-01-03,2,6,2024-04-30 13:07:13.382187,postgres,False,NaN
2,3,523,4,2016-01-02,2016-01-05,2016-01-03,2,7,2024-04-30 13:07:13.382187,postgres,False,NaN
3,4,175,4,2016-01-03,2016-01-04,2016-01-05,1,3,2024-04-30 13:07:13.382187,postgres,True,1.0
4,5,1324,4,2016-01-03,2016-01-06,2016-01-06,2,6,2024-04-30 13:07:13.382187,postgres,False,NaN


In [48]:
orders['weekend'] = orders['shipped_date'].dt.dayofweek.isin([6, 0])


In [49]:
orders.head()

,order_id,customer_id,order_status,order_date,required_date,shipped_date,store_id,staff_id,extract_time,source,late_delivery,latency_days,weekend
0,1,259,4,2016-01-01,2016-01-03,2016-01-03,1,2,2024-04-30 13:07:13.382187,postgres,False,NaN,True
1,2,1212,4,2016-01-01,2016-01-04,2016-01-03,2,6,2024-04-30 13:07:13.382187,postgres,False,NaN,True
2,3,523,4,2016-01-02,2016-01-05,2016-01-03,2,7,2024-04-30 13:07:13.382187,postgres,False,NaN,True
3,4,175,4,2016-01-03,2016-01-04,2016-01-05,1,3,2024-04-30 13:07:13.382187,postgres,True,1.0,False
4,5,1324,4,2016-01-03,2016-01-06,2016-01-06,2,6,2024-04-30 13:07:13.382187,postgres,False,NaN,False


In [50]:
import os

# Define the parent directory path
parent_directory = '../staging_2'

# Define the subdirectory path
subdirectory = 'orders'

# Create the parent directory if it doesn't exist
if not os.path.exists(parent_directory):
    os.makedirs(parent_directory)
    print(f"Parent directory '{parent_directory}' created.")

# Create the subdirectory if it doesn't exist
if not os.path.exists(os.path.join(parent_directory, subdirectory)):
    os.makedirs(os.path.join(parent_directory, subdirectory))
    print(f"Subdirectory '{subdirectory}' created.")

# Define the file path where you want to save the CSV file
file_path = os.path.join(parent_directory, subdirectory, 'orders.csv')

# Save the DataFrame as a CSV file
orders.to_csv(file_path, index=False)

print(f"CSV file saved successfully in the '{subdirectory}' subfolder within the '{parent_directory}' folder.")


CSV file saved successfully in the 'orders' subfolder within the '../staging_2' folder.


In [51]:
customers=pd.read_csv('../staging_1/customers/customers.csv')

In [52]:
stores=pd.read_csv('../staging_1/stores/stores.csv')

In [53]:
stores.head()

,store_id,store_name,phone,email,street,city,state,zip_code,Extract Time,Source
0,1,Santa Cruz Bikes,(831) 476-4321,NaN,3700 Portola Drive,Santa Cruz,CA,NaN,2024-04-30 13:25:01.164185,datalake
1,2,Baldwin Bikes,(516) 379-8888,baldwin@bikes.shop,4200 Chestnut Lane,Baldwin,NY,11432.0,2024-04-30 13:25:01.164185,datalake
2,3,Rowlett Bikes,(972) 530-5555,rowlett@bikes.shop,8000 Fairway Avenue,Rowlett,TX,75088.0,2024-04-30 13:25:01.164185,datalake


In [54]:

# Merge customers data with stores data on the city column
merged_customer_df = pd.merge(customers, stores[['city','store_name']], on='city', how='left', suffixes=('', '_store'))

merged_customer_df['local'] = ~merged_customer_df['store_name'].isnull()

merged_customer_df.shape


(1445, 12)

In [55]:
merged_customer_df.sort_values(by='city', inplace=True)
merged_customer_df.head(100)[50:]


,customer_id,first_name,last_name,email,street,city,state,zip_code,Extract Time,Source,store_name,local
778,779,Major,Merrill,major.merrill@aol.com,78 Thatcher Court,Astoria,NY,11102,2024-04-30 13:25:01.229172,datalake,NaN,False
495,496,Shu,Mays,shu.mays@gmail.com,996 St Margarets Lane,Astoria,NY,11102,2024-04-30 13:25:01.229172,datalake,NaN,False
1415,1416,Miranda,Kennedy,miranda.kennedy@gmail.com,726 East High St.,Astoria,NY,11102,2024-04-30 13:25:01.229172,datalake,NaN,False
820,821,Cassidy,Clark,cassidy.clark@hotmail.com,9879 Old Kingston Road,Astoria,NY,11102,2024-04-30 13:25:01.229172,datalake,NaN,False
37,38,Zelma,Browning,zelma.browning@aol.com,296 Second Street,Astoria,NY,11102,2024-04-30 13:25:01.229172,datalake,NaN,False
1286,1287,Victor,Pittman,victor.pittman@aol.com,9294 Devon Dr.,Astoria,NY,11102,2024-04-30 13:25:01.229172,datalake,NaN,False
734,735,Aide,Franco,aide.franco@msn.com,8017 Lake Forest St.,Atwater,CA,95301,2024-04-30 13:25:01.229172,datalake,NaN,False
150,151,Joesph,Delacruz,joesph.delacruz@aol.com,184 Cypress Street,Atwater,CA,95301,2024-04-30 13:25:01.229172,datalake,NaN,False
252,253,Maurice,Norton,maurice.norton@hotmail.com,32 Mayflower Dr.,Atwater,CA,95301,2024-04-30 13:25:01.229172,datalake,NaN,False
23,24,Corene,Wall,corene.wall@msn.com,9601 Ocean Rd.,Atwater,CA,95301,2024-04-30 13:25:01.229172,datalake,NaN,False


In [56]:
merged_customer_df.drop(columns=['store_name'], inplace=True)

In [57]:
merged_customer_df

,customer_id,first_name,last_name,email,street,city,state,zip_code,Extract Time,Source,local
1113,1114,Douglass,Blankenship,douglass.blankenship@hotmail.com,409 W. Rock Maple Lane,Albany,NY,12203,2024-04-30 13:25:01.229172,datalake,False
344,345,Mi,Gray,mi.gray@aol.com,27 Roehampton Dr.,Albany,NY,12203,2024-04-30 13:25:01.229172,datalake,False
209,210,Priscilla,Wilkins,priscilla.wilkins@aol.com,653 Pawnee Rd.,Albany,NY,12203,2024-04-30 13:25:01.229172,datalake,False
598,599,Jonell,Rivas,jonell.rivas@msn.com,8415 East High Point St.,Amarillo,TX,79106,2024-04-30 13:25:01.229172,datalake,False
729,730,Andria,Rivers,andria.rivers@aol.com,183 Ohio Ave.,Amarillo,TX,79106,2024-04-30 13:25:01.229172,datalake,False
...,...,...,...,...,...,...,...,...,...,...,...
771,772,Latricia,Lindsey,latricia.lindsey@hotmail.com,288 Campfire Dr.,Yorktown Heights,NY,10598,2024-04-30 13:25:01.229172,datalake,False
993,994,Demarcus,Reese,demarcus.reese@gmail.com,8087 Armstrong Dr.,Yorktown Heights,NY,10598,2024-04-30 13:25:01.229172,datalake,False
616,617,Jenna,Saunders,jenna.saunders@aol.com,4 Young St.,Yorktown Heights,NY,10598,2024-04-30 13:25:01.229172,datalake,False
1298,1299,Shauna,Edwards,shauna.edwards@msn.com,8 Dogwood Ave.,Yorktown Heights,NY,10598,2024-04-30 13:25:01.229172,datalake,False


In [58]:
import os

# Define the parent directory path
parent_directory = '../staging_2'

# Define the subdirectory path
subdirectory = 'customers'

# Create the parent directory if it doesn't exist
if not os.path.exists(parent_directory):
    os.makedirs(parent_directory)
    print(f"Parent directory '{parent_directory}' created.")

# Create the subdirectory if it doesn't exist
if not os.path.exists(os.path.join(parent_directory, subdirectory)):
    os.makedirs(os.path.join(parent_directory, subdirectory))
    print(f"Subdirectory '{subdirectory}' created.")

# Define the file path where you want to save the CSV file
file_path = os.path.join(parent_directory, subdirectory, 'customers.csv')

# Save the DataFrame as a CSV file
merged_customer_df.to_csv(file_path, index=False)

print(f"CSV file saved successfully in the '{subdirectory}' subfolder within the '{parent_directory}' folder.")


CSV file saved successfully in the 'customers' subfolder within the '../staging_2' folder.


In [59]:
# Define the order status mapping
status_mapping = {
    1: 'Pending',
    2: 'Processing',
    3: 'Rejected',
    4: 'Completed'
}

# Define the path to save the CSV file
csv_path = '../staging_2/order_status_lookup/order_status_lookup.csv'

# Write the status mapping to a CSV file
with open(csv_path, 'w') as file:
    file.write("order_status,status_description\n")
    for status_code, status_description in status_mapping.items():
        file.write(f"{status_code},{status_description}\n")
